In [1]:
import warnings

from typing import List

warnings.filterwarnings("ignore")

In [2]:
# Dejar el path principal como el anterior.
import sys
sys.path.append('../')

Seteamos las configuraciones del Logging

In [3]:
import logging

# Crear el logger
log = logging.getLogger(__name__)

# Setear el nivel del registro
log.setLevel(logging.DEBUG)

# Formato de los mensajes
formatter = logging.Formatter("%(levelname)s: (%(asctime)s) [%(filename)s: %(lineno)s] %(message)s")

if not log.hasHandlers():
    # Handlers
    file_handler = logging.FileHandler("logging.log")
    file_handler.setFormatter(formatter)  # Setear el formato del handler
    # Agregar el handler al logger
    # log.addHandler(file_handler)

    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)
    # Agregar el handler al logger
    log.addHandler(stream_handler)

Instalar la librería plot-likert y otras librerías útiles

In [4]:
# Librería para hacer gráficos Likert
# !pip install plot-likert

# Para obtener datos de excel
# !pip install openpyxl

# Para tener un transformador de data
# !pip install -U scikit-learn

# Para tener herramientas estadísticas
# !python -m pip install statsmodels

# Para tener Seaborn
# !pip install seaborn

Empezamos importando la librería para verificar que estuvo bien instalada.

In [5]:
import plot_likert

Importamos algunas librerías útiles para el resto del notebook

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [7]:
DATA_PATH = Path(r"..\data")

if not DATA_PATH.exists():
    log.info(f"Creando carpeta {DATA_PATH = }")
    DATA_PATH.mkdir()

In [8]:
QNA_COMPLETE_PATH = DATA_PATH / "questions-and-answers-complete.xlsx"
QNA_COMPLETE_PATH

WindowsPath('../data/questions-and-answers-complete.xlsx')

In [9]:
log.debug(f"Importando datos de {QNA_COMPLETE_PATH}")
df = pd.read_excel(QNA_COMPLETE_PATH)

df.head()

DEBUG: (2023-01-07 19:22:03,771) [3058811912.py: 1] Importando datos de ..\data\questions-and-answers-complete.xlsx


,Prom Cs,¿Los profesores de ciencias consideran los intereses de los alumnos para elaborar y organizar las evaluaciones?,De las actividades realizadas en clase ¿el profesor considera y respeta nuestros intereses?,¿El profesor retroalimenta y refuerza en todas las clases lo que hemos ido aprendiendo?,El profesor de la clase de ciencias ¿anota y explica el objetivo que se va a trabajar durante la clase?,"El profesor, al cierre de su clase ¿realiza preguntas para verificar si hemos aprendido?",Cuando el profesor da tareas ¿las explica claramente y nos da ejemplos que orientan para realizarlas?,Durante la actividad ¿el profesor circula explicando y resolviendo dudas?,"¿Se realizan salidas a terreno: museos, zoológicos y otros lugares que ayudan al aprendizaje?","Para hacer las clases, el profesor ¿nos lleva a otros espacios motivantes para que aprendamos mejor?",...,"Cuando me queda alguna duda de la clase, ¿me dirijo confiadamente al profesor a preguntar?","Cuando el profesor es cercano conmigo y mis compañeros, ¿nos da confianza y seguridad para preguntar todas las dudas?",Cuando tengo más cercanía y confianza con el profesor ¿aprendo mejor?,"¿Los profesores de ciencias ofrecen diversas actividades grupales de un mismo tema, para que los estudiantes, elijan en la que quieren trabajar?",¿Los profesores de ciencias integran a todos los estudiantes para que participen en las actividades grupales dentro de la sala de clases?,"¿Los profesores de ciencias integran a todos los estudiantes en grupos, para que participen en las actividades de investigación, fuera del colegio?","¿Los profesores, muestran interés por reforzar a los alumnos que más les cuesta?","Cuando el profesor nos hace realizar actividades variadas, en ciencias ¿comprendo mejor la materia?",¿Los profesores de ciencias integran a todos los estudiantes para que participen en las actividades prácticas de laboratorio?,"En las clases de ciencias, ¿algunos alumnos son evaluados con carpetas de actividades realizadas en el colegio y fuera de él?"
0,6.8,Ocasionalmente,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases",Nunca,Rara vez,...,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,Casi siempre,Rara vez,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases",Nunca
1,6.7,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases"
2,6.1,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente,"Siempre, en todas las clases","Siempre, en todas las clases",Nunca,Ocasionalmente,...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Nunca,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente
3,4.6,Casi siempre,Casi siempre,Rara vez,Casi siempre,Casi siempre,Casi siempre,Ocasionalmente,Nunca,Ocasionalmente,...,Nunca,Casi siempre,"Siempre, en todas las clases",Ocasionalmente,Casi siempre,"Siempre, en todas las clases",Rara vez,Ocasionalmente,Rara vez,Nunca
4,6.5,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,Nunca,Ocasionalmente,...,"Siempr

In [10]:
questions = list(df.columns)[1:]
questions

['¿Los profesores de ciencias consideran los intereses de los alumnos para elaborar y organizar las evaluaciones?',
 'De las actividades realizadas en clase ¿el profesor considera y respeta nuestros intereses?',
 '¿El profesor retroalimenta y refuerza en todas las clases lo que hemos ido aprendiendo?',
 'El profesor de la clase de ciencias ¿anota y explica el objetivo que se va a trabajar durante la clase?',
 'El profesor, al cierre de su clase ¿realiza preguntas para verificar si hemos aprendido?',
 'Cuando el profesor da tareas ¿las explica claramente y nos da ejemplos que orientan para realizarlas?',
 'Durante la actividad ¿el profesor circula explicando y resolviendo dudas?',
 '¿Se realizan salidas a terreno: museos, zoológicos y otros lugares que ayudan al aprendizaje?',
 'Para hacer las clases, el profesor ¿nos lleva a otros espacios motivantes para que aprendamos mejor?',
 'El profesor de ciencias ¿presenta en sus clases material atractivo para aprender?',
 '¿Los profesores comp

In [11]:
import copy


log.debug("Renombrando columnas")
questions_copy = copy.copy(questions)
for i in range(len(questions)):
    # questions_copy[i] = f"P{i+1}.: " + questions[i]
    questions_copy[i] = f"P{i+1}"

df.columns = questions = ["Notas"] +  questions_copy


df.head()

DEBUG: (2023-01-07 19:22:04,175) [2314446844.py: 4] Renombrando columnas


,Notas,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31
0,6.8,Ocasionalmente,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases",Nunca,Rara vez,...,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,Casi siempre,Rara vez,Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases",Nunca
1,6.7,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases"
2,6.1,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente,"Siempre, en todas las clases","Siempre, en todas las clases",Nunca,Ocasionalmente,...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Nunca,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente
3,4.6,Casi siempre,Casi siempre,Rara vez,Casi siempre,Casi siempre,Casi siempre,Ocasionalmente,Nunca,Ocasionalmente,...,Nunca,Casi siempre,"Siempre, en todas las clases",Ocasionalmente,Casi siempre,"Siempre, en todas las clases",Rara vez,Ocasionalmente,Rara vez,Nunca
4,6.5,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Casi siempre,Nunca,Ocasionalmente,...,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Ocasionalmente,"Siempre, en todas las clases","Siempre, en todas las clases","Siempre, en todas las clases",Nunca


In [12]:
scales = [
    "Nunca",
    "Rara vez",
    "Ocasionalmente",
    "Casi siempre",
    "Siempre, en todas las clases"
]

In [13]:
from collections import OrderedDict
df_enc = OrderedDict()

encoder = {name: i-2 for i, name in enumerate(scales)}
for col_name in questions_copy:
    df_enc[col_name] = df[col_name].map(encoder)

df_enc["Notas"] = (df["Notas"] >= 6).astype(int)

df_enc = pd.DataFrame(df_enc)
df_enc

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P23,P24,P25,P26,P27,P28,P29,P30,P31,Notas
0,0,1,2,2,2,1,2,-2,-1,1,...,2,2,1,1,-1,1,2,2,-2,1
1,2,2,2,2,1,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,1
2,2,2,2,2,0,2,2,-2,0,2,...,2,2,2,2,-2,2,2,2,0,1
3,1,1,-1,1,1,1,0,-2,0,0,...,1,2,0,1,2,-1,0,-1,-2,0
4,2,2,2,2,2,2,1,-2,0,2,...,2,2,2,2,0,2,2,2,-2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,1,2,1,2,1,1,1,-1,1,2,...,1,1,1,1,1,2,1,1,1,0
82,0,-1,1,2,0,2,-1,-1,-1,-2,...,-1,-1,0,0,0,0,-2,-2,-2,0
83,2,0,1,2,2,0,2,-2,-1,2,...,1,-1,0,2,1,2,1,2,-2,1
84,2,2,2,2,2,2,2,-1,-1,2,...,1,2,1,2,2,2,2,2,0,1


In [14]:
df["Notas"] >= 6

0      True
1      True
2      True
3     False
4      True
      ...  
81    False
82    False
83     True
84     True
85    False
Name: Notas, Length: 86, dtype: bool

In [15]:
df_enc.describe(percentiles=[i / 10 for i in range(10)])

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P23,P24,P25,P26,P27,P28,P29,P30,P31,Notas
count,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,...,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000
mean,0.686047,1.023256,1.360465,1.662791,1.034884,1.174419,1.302326,-1.639535,-1.093023,0.837209,...,1.383721,1.139535,0.500000,1.279070,-0.046512,1.162791,0.953488,0.279070,-0.988372,0.453488
std,1.031978,0.920061,0.866539,0.661735,0.975556,1.019710,0.946446,0.880011,1.047438,1.125746,...,1.007835,1.097192,1.317306,0.862979,1.637121,0.931292,1.072731,1.606246,1.231873,0.500752
min,-2.000000,-1.000000,-2.000000,-1.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,0.000000
0%,-2.000000,-1.000000,-2.000000,-1.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,0.000000
10%,-1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-2.000000,-2.000000,-1.000000,...,0.000000,-0.500000,-1.000000,0.000000,-2.000000,0.000000,0.000000,-2.000000,-2.000000,0.000000
20%,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,-2.000000,-2.000000,0.000000,...,1.000000,0.000000,-1.000000,1.000000,-2.000000,0.000000,0.000000,-2.000000,-2.000000,0.000000
30%,0.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,-2.000000,-2.000000,0.000000,...,1.000000,1.000000,0.000000,1.000000,-1.500000,1.000000,0.000000,-1.000000,-2.000000,0.000000
40%,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,-2.000000,-2.000000,1.000000,...,2.000000,1.000000,0.000000,1.000000,-1.000000,1.000000,1.000000,0.000000,-2.000000,0.000000
50%,1.000000,1.000000,2.000000,2.000000,1.000000,2.000000,2.000000,-2.000000,-1.000000,1.000000,...,2.000000,2.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,-1.000000,0.000000


In [16]:
import statsmodels.api as sm

In [17]:
df_enc.endog = np.array(df_enc.pop("Notas")).reshape((-1, 1))

# df_enc.endog

In [18]:
np.mean(df_enc.endog)

0.45348837209302323

In [19]:
df_enc.exog = sm.add_constant(df_enc)
df_enc.exog

,const,P1,P2,P3,P4,P5,P6,P7,P8,P9,...,P22,P23,P24,P25,P26,P27,P28,P29,P30,P31
0,1.0,0,1,2,2,2,1,2,-2,-1,...,1,2,2,1,1,-1,1,2,2,-2
1,1.0,2,2,2,2,1,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2,1.0,2,2,2,2,0,2,2,-2,0,...,2,2,2,2,2,-2,2,2,2,0
3,1.0,1,1,-1,1,1,1,0,-2,0,...,-2,1,2,0,1,2,-1,0,-1,-2
4,1.0,2,2,2,2,2,2,1,-2,0,...,2,2,2,2,2,0,2,2,2,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,1.0,1,2,1,2,1,1,1,-1,1,...,1,1,1,1,1,1,2,1,1,1
82,1.0,0,-1,1,2,0,2,-1,-1,-1,...,-2,-1,-1,0,0,0,0,-2,-2,-2
83,1.0,2,0,1,2,2,0,2,-2,-1,...,2,1,-1,0,2,1,2,1,2,-2
84,1.0,2,2,2,2,2,2,2,-1,-1,...,1,1,2,1,2,2,2,2,2,0


In [20]:
logit_mod = sm.Logit(df_enc.endog, df_enc.exog)

logit_res = logit_mod.fit()

logit_res.summary()

Optimization terminated successfully.
         Current function value: 0.462160
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                   86
Model:                          Logit   Df Residuals:                       54
Method:                           MLE   Df Model:                           31
Date:                Sat, 07 Jan 2023   Pseudo R-squ.:                  0.3291
Time:                        19:22:05   Log-Likelihood:                -39.746
converged:                       True   LL-Null:                       -59.238
Covariance Type:            nonrobust   LLR p-value:                    0.1536
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.1324      1.478     -0.766      0.443      -4.029       1.764
P1             0.4598      0.585      0.787      0.431      -0.686       1.605
P2            -0.3634      0.492     -0.739      0.460      -1.327       0.600
P3             0.1165      0.679      0.172      0.864      -1.214       1.447
P4            -0.5987      0.681     -0.879      0.379      -1.933       0.736
P5             0.5610      0.486      1.153      0.249      -0.392       1.514
P6            -0.2358      0.578     -0.408      0.683      -1.368       0.897
P7            -0.2223      0.649     -0.343      0.732      -1.494       1.050
P8             0.1734      0.530      0.327      0.744      -0.866       1.213
P9             0.2850      0.495      0.576      0.565      -0.686       1.256
P10           -0.7611      0.538     -1.415      0.157      -1.816       0.293
P11            1.0552      0.943      1.119      0.263      -0.793       2.903
P12           -1.0015      0.611     -1.640      0.101      -2.199       0.196
P13           -0.3943      0.956     -0.412      0.680      -2.269       1.480
P14           -0.0789      0.284     -0.278      0.781      -0.635       0.477
P15           -0.7346      0.393     -1.870      0.062      -1.505       0.036
P16           -0.4362      0.380     -1.148      0.251      -1.181       0.309
P17           -0.4920      0.415     -1.185      0.236      -1.306       0.322
P18            0.2858      0.601      0.476      0.634      -0.891       1.463
P19           -0.0926      0.533     -0.174      0.862      -1.137       0.952
P20           -0.3602      0.321     -1.124      0.261      -0.988       0.268
P21            1.3964      0.713      1.959      0.050      -0.001       2.794
P22           -0.0317      0.512     -0.062      0.951      -1.034       0.971
P23            1.8159      0.810      2.242      0.025       0.229       3.403
P24           -0.8041      0.549     -1.465      0.143      -1.880       0.271
P25           -0.4855      0.368     -1.320      0.187      -1.206       0.235
P26           -0.3879      0.555     -0.699      0.484      -1.475       0.699
P27           -0.1288      0.282     -0.456      0.648      -0.682       0.425
P28           -0.2749      0.685     -0.401      0.688      -1.618       1.068
P29            1.0087      0.588      1.716      0.086      -0.143       2.161
P30            0.8884      0.299      2.971      0.003       0.302       1.475
P31            0.6624      0.466      1.421      0.155      -0.251       1.576
==============================================================================
"""

In [27]:
from sklearn.metrics import mean_squared_error
# ((logit_res.predict(df_enc.exog) - df_enc.endog.reshape(-1,)) ** 2).sum() / len(df_enc.endog.reshape(-1,))
# df_enc.endog.reshape(-1,)
mean_squared_error(
    y_true=df_enc.endog.reshape(-1,),
    y_pred=logit_res.predict(df_enc.exog),
)

0.15437325951805322

In [28]:

alumnos = np.random.choice(
    a=len(df_enc.exog),
    size=5
)
alumnos

array([20, 81, 10, 36, 39])

In [30]:
logit_res.predict(df_enc.exog.iloc[alumnos])

20    0.146807
81    0.169136
10    0.734837
36    0.328033
39    0.271617
dtype: float64

In [32]:
df_enc.endog.reshape(-1,)[alumnos]

array([1, 0, 1, 0, 0])